## Рекомендательная система фильмов.

In [1]:
# Подключение библиотек.
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Загрузка данных.

In [2]:
films = pd.read_csv('netflix_data.csv')

In [3]:
films

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [4]:
films.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [5]:
# Создание нового DataFrame для удобства.
data = films[['title', 'director', 'cast', 'listed_in', 'description']]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        8807 non-null   object
 1   director     6173 non-null   object
 2   cast         7982 non-null   object
 3   listed_in    8807 non-null   object
 4   description  8807 non-null   object
dtypes: object(5)
memory usage: 344.2+ KB


In [6]:
# Заполнение строк с пропусками
data['director'].fillna('Unknown', inplace=True)
data['cast'].fillna('Unknown', inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        8807 non-null   object
 1   director     8807 non-null   object
 2   cast         8807 non-null   object
 3   listed_in    8807 non-null   object
 4   description  8807 non-null   object
dtypes: object(5)
memory usage: 344.2+ KB


C:\Users\Я\AppData\Local\Temp\ipykernel_3672\3307699918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['director'].fillna('Unknown', inplace=True)
C:\Users\Я\AppData\Local\Temp\ipykernel_3672\3307699918.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cast'].fillna('Unknown', inplace=True)


### Обработка данных.

In [7]:
# Перевод в нижний регистр и удаление ненужных символов.
data['listed_in'] = data['listed_in'].str.lower()
data['listed_in'] = data['listed_in'].apply(lambda x: re.sub("[\W_]+", " ", x).strip())

C:\Users\Я\AppData\Local\Temp\ipykernel_3672\192299818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['listed_in'] = data['listed_in'].str.lower()
C:\Users\Я\AppData\Local\Temp\ipykernel_3672\192299818.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['listed_in'] = data['listed_in'].apply(lambda x: re.sub("[\W_]+", " ", x).strip())


In [8]:
# Объединение всех полей для последующих рекомендаций.
data['unification'] = data['title'] + ' ' + data['director'] + ' ' + data['cast'] + ' ' + data['listed_in'] + ' ' + data['description']

# Обработка нового столбца.
data['unification'] = data['unification'].str.lower()
data['unification'] = data['unification'].apply(lambda x: re.sub("[\W_]+", " ", x).strip())

C:\Users\Я\AppData\Local\Temp\ipykernel_3672\935281221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['unification'] = data['title'] + ' ' + data['director'] + ' ' + data['cast'] + ' ' + data['listed_in'] + ' ' + data['description']
C:\Users\Я\AppData\Local\Temp\ipykernel_3672\935281221.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['unification'] = data['unification'].str.lower()
C:\Users\Я\AppData\Local\Temp\ipykernel_3672\935281221.py:6: SettingWithCopyWarning: 
A value is trying to b

In [9]:
data.head()

,title,director,cast,listed_in,description,unification
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,documentaries,"As her father nears the end of his life, filmm...",dick johnson is dead kirsten johnson unknown d...
1,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",international tv shows tv dramas tv mysteries,"After crossing paths at a party, a Cape Town t...",blood water unknown ama qamata khosi ngema gai...
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",crime tv shows international tv shows tv actio...,To protect his family from a powerful drug lor...,ganglands julien leclercq sami bouajila tracy ...
3,Jailbirds New Orleans,Unknown,Unknown,docuseries reality tv,"Feuds, flirtations and toilet talk go down amo...",jailbirds new orleans unknown unknown docuseri...
4,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",international tv shows romantic tv shows tv co...,In a city of coaching centers known to train I...,kota factory unknown mayur more jitendra kumar...


#### Список топ-10 популярных фильмов.

Нет столбца с рейтингом... Но можно просто рекомендовать 10 случайных фильмов/сериалов.

In [10]:
def get_top_10(data):
    return data.sample(10)['title'].reset_index(drop=True)

In [11]:
# Проверка.
print(get_top_10(data))

0                                     Harud
1                          Puriyatha Puthir
2                                    V Wars
3                       The Kissing Booth 3
4               What's Eating Gilbert Grape
5       Lalbaug Parel: Zali Mumbai Sonyachi
6                        World at Your Feet
7                                 Moonlight
8                              Silver Spoon
9    Chhota Bheem And The Crown of Valhalla
Name: title, dtype: object


#### Рекомендации по жанру.

In [12]:
# Преобразование.
tfidf_genre_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_genre_matrix = tfidf_genre_vectorizer.fit_transform(data['listed_in'])

In [13]:
# Косинусное сходство.
genre_cosine_sim = cosine_similarity(tfidf_genre_matrix, tfidf_genre_matrix)
genre_cosine_sim.shape

(8807, 8807)

In [14]:
def recommend_by_genre(genre, cosine_sim=genre_cosine_sim):
    genre_indices = data[data['listed_in'].str.contains(genre, case=False)].index.to_list()
    
    if not genre_indices:
        return 'Жанр не найден.'
    
    sim_scores = list(enumerate(cosine_sim[genre_indices[0]]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]

    recommendations = []
    for i, _ in sim_scores:
        title = data.iloc[i]['title']
        genres = data.iloc[i]['listed_in']
        recommendations.append(f"{title}. Жанры: {genres}")

    return recommendations

In [16]:
# Проверка.
genre = 'action'
print(f"Рекомендации по жанру '{genre}':")
recommendations = recommend_by_genre(genre)
for rec in recommendations:
    print(rec)

Рекомендации по жанру 'action':
Bangkok Breaking. Жанры: crime tv shows international tv shows tv action adventure
Undercover. Жанры: crime tv shows international tv shows tv action adventure
Lupin. Жанры: crime tv shows international tv shows tv action adventure
Dealer. Жанры: crime tv shows international tv shows tv action adventure
Fauda. Жанры: crime tv shows international tv shows tv action adventure
Nowhere Man. Жанры: crime tv shows international tv shows tv action adventure
Chosen. Жанры: crime tv shows international tv shows tv action adventure
The Eagle of El-Se'eed. Жанры: crime tv shows international tv shows tv action adventure
Monkey Twins. Жанры: crime tv shows international tv shows tv action adventure
Smoking. Жанры: crime tv shows international tv shows tv action adventure


#### Рекомендации по контенту.

In [17]:
# Преобразование.
tfidf_content_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_content_matrix = tfidf_genre_vectorizer.fit_transform(data['unification'])

In [18]:
# Косинусное сходство.
content_cosine_sim = cosine_similarity(tfidf_content_matrix, tfidf_content_matrix)
content_cosine_sim.shape

(8807, 8807)

In [19]:
def recommend_by_content(title, cosine_sim=content_cosine_sim):
    idx = data[data['title'].str.lower() == title.lower()].index[0]

    if not idx:
        return 'Фильм не найден.'

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
    
    recommendations = []
    for i, score in sim_scores:
        recommendations.append(f"{data.iloc[i]['title']} (Косинусное сходство: {score:.4f})")

    return recommendations

In [22]:
# Проверка.
title = 'SHAMAN KING'
print(f"Рекомендации для '{title}':")
recommendations = recommend_by_content(title)
for rec in recommendations:
    print(rec)

Рекомендации для 'SHAMAN KING':
Magi: Adventure of Sinbad (Косинусное сходство: 0.1815)
Fullmetal Alchemist: Brotherhood (Косинусное сходство: 0.1750)
Little Witch Academia (Косинусное сходство: 0.1407)
JoJo's Bizarre Adventure (Косинусное сходство: 0.1321)
Neon Genesis Evangelion (Косинусное сходство: 0.1272)
EVANGELION: DEATH (TRUE)² (Косинусное сходство: 0.1263)
Violet Evergarden: Special (Косинусное сходство: 0.1251)
Forest of Piano (Косинусное сходство: 0.1231)
Record of Grancrest War (Косинусное сходство: 0.1207)
7SEEDS (Косинусное сходство: 0.1142)


In [21]:
data.to_csv('data.csv')